# Arbeitspaket (AP) 2: Management & Nutzung Temporale Daten

### Persönliche Angaben (bitte ergänzen)

<table>
  <tr>
    <td>Vorname:</td>
    <td>Phil</td>
  </tr>
  <tr>
    <td>Nachname:</td>
    <td>Frei</td>
  </tr>
  <tr>
    <td>Immatrikulationsnummer:</td>
    <td>21-548-946</td>
  </tr>
  <tr>
    <td>Modul:</td>
    <td>Data Science</td>
  </tr>
  <tr>
    <td>Prüfungsdatum / Raum / Zeit:</td>
    <td>06.10.2025 / Raum: MU O2.001 / 8:00 – 11:45</td>
  </tr>
  <tr>
    <td>Erlaubte Hilfsmittel:</td>
    <td>w.MA.XX.DS.25HS (Data Science)<br>Open Book, Eigener Computer, Internet-Zugang</td>
  </tr>
  <tr>
  <td>Nicht erlaubt:</td>
  <td>Nicht erlaubt ist der Einsatz beliebiger Formen von generativer KI (z.B. Copilot, ChatGPT) <br> sowie beliebige Formen von Kommunikation oder Kollaboration mit anderen Menschen.</td>
</tr>
</table>

## Bewertungskriterien

### <b style="color: gray;">(max. erreichbare Punkte: 40)</b>

<table>
  <thead>
    <tr>
      <th>Kategorie</th>
      <th>Beschreibung</th>
      <th>Punkteverteilung</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Code nicht lauffähig oder Ergebnisse nicht sinnvoll</td>
      <td>Der Code enthält Fehler, die verhindern, dass er ausgeführt werden kann (z.B. Syntaxfehler) oder es werden Ergebnisse ausgegeben, welche nicht zur Fragestellung passen.</td>
      <td>0 Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit gravierenden Mängeln</td>
      <td>Der Code läuft, aber die Ergebnisse sind aufgrund wesentlicher Fehler unvollständig (z.B. fehlende Joins, gravierende Fehler in SQL-Abfragen). Nur geringer Fortschritt erkennbar.</td>
      <td>25% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit mittleren Mängeln</td>
      <td>Der Code läuft und liefert teilweise korrekte Ergebnisse, aber es gibt grössere Fehler (z.B. fehlende Spalten, unvollständige SQL-Abfragen). Die Ergebnisse sind nachvollziehbar, aber unvollständig oder ungenau.</td>
      <td>50% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit minimalen Mängeln</td>
      <td>Der Code läuft und liefert ein weitgehend korrektes Ergebnis, aber kleinere Fehler (z.B. falsche oder fehlende Sortierung, Rundung von Werten falsch) beeinträchtigen die Vollständigkeit des Ergebnisses.</td>
      <td>75% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig und korrekt</td>
      <td>Der Code läuft einwandfrei und liefert das korrekte Ergebnis ohne Mängel.</td>
      <td>100% der max. erreichbaren Punkte</td>
    </tr>
  </tbody>
</table>



## <b>Vorbereitung (Dieser Teil wird <u>nicht</u> bewertet!)</b>

#### <b>Beschreibung des Datensatzes `synthetic_ecommerce_orders.csv`</b>

- **Inhalt:** Synthetische E-Commerce-Bestellereignisse (April–Juni 2024)
- **Schema:**
  - `time` (lokale Zeit, Europe/Zurich, Format `YYYY-MM-DD HH:MM:SS`)
  - `customer_id` (int)
  - `product_id` (int)
  - `price` (float, EUR)
  - `discount` (float, EUR)
  - `status` (category: `placed`, `paid`, `cancelled`, `refunded`)
- **Hinweise zur Nutzung:**
  - Für **Abfragen** mit Tinyflux Zeitstempel nach **UTC** konvertieren.
  - `price - discount` kann als **Nettoerlös** interpretiert werden (v. a. für `paid`).
  - Leichte Tages-/Wochenmuster (mehr Aktivität 09:00–21:00, werktags).
- **Ziel:** Einlesen, in Tinyflux schreiben (Fields: `price`, `discount`; Tags: `customer_id`, `product_id`, `status`, optional `channel`), Zeitfenster-Abfragen und Aggregationen.


#### <b>1.) Python Libraries und Settings importieren:</b>

In [88]:
# Libraries
import os
import pandas as pd
import csv
from tinyflux import TinyFlux, Point, FieldQuery, TagQuery, TimeQuery
from datetime import datetime, timezone, timedelta

# Settings
import warnings
warnings.filterwarnings("ignore")

# Abkürzungen für Query-Typen
time = TimeQuery()
tags = TagQuery()
field = FieldQuery()

#### <b>2.) Funktion für die Erstellung eines Dataframes aus Query-Resultaten in den Arbeitsspeicher laden:</b>

In [89]:
# Define the function to transform the list of Point objects into a DataFrame
def points_to_dataframe(points_list):
    data = []
    
    # Extract the relevant information
    for point in points_list:
        data.append({
            "time": point.time,  # Access time attribute
            "customer_id": point.tags['customer_id'],  # Access customer_id from tags dictionary
            "product_id": point.tags['product_id'],  # Access product_id from tags dictionary
            "status": point.tags['status'],  # Access status from tags dictionary
            "price": point.fields['price'],  # Access price from fields dictionary
            "discount": point.fields['discount']  # Access discount from fields dictionary
        })

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Convert time to datetime
    df['time'] = pd.to_datetime(df['time'])

    return df

#### <b>3.) Starten Sie eine GitHub Codespaces Instanz auf Basis Ihres Forks des folgenden GitHub Repositories:</b>

##### GitHub-Repository: https://github.com/MariaPelli/tinyflux


#### <b>4.) Erstellen Sie mit Tinyflux eine Datenbank mit dem Namen "ecommerce.db".</b>

In [90]:
# Erstellung der Tinyflux-Datenbank
db = TinyFlux("ecommerce.db")

## <b>Aufgaben: Einlesen der Daten in die Datenbank (Dieser Teil wird bewertet!)</b>

#### <b>Aufgabe (1): Lesen Sie die CSV-Datei 'synthetic_ecommerce_transactions.csv' in ein Dataframe mit dem Namen "df" ein, zeigen Sie die ersten 10 Records und geben deskriptive Statistiken des Dataframes aus.</b>

<b>Details zur Aufgabenstellung:</b>

- Das Laden der CSV-Datei kann mittels der Pandas-Funktion "read_csv" (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) erfolgen
- Achten Sie darauf, den Zeitstempel mittels der Pandas-Funktion __df['timestamp']= pd.to_datetime(df['timestamp'], format='mixed')__ auf das Format datetime zu setzen (https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)
- Die ersten 10 Records können mittels der Pandas-Funktion "df.head()" wiedergegeben werden (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html) 
- Deskriptive Statistiken können mittels Pandas-Funktion "df.describe()" (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) wiedergegeben werden. Geben Sie diese für quantitative und kategoriale Variablen aus, indem Sie den Parameter include auf 'all' setzen.

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [91]:
# CSV-Datei in Dataframe laden
df = pd.read_csv('synthetic_ecommerce_orders.csv')
# Das Format des Zeitstempels in Pandas setzen
df['time']= pd.to_datetime(df['time'], format='mixed')

In [92]:
# Erste 10 Records des Dataframes anzeigen
df.head(10)

,time,customer_id,product_id,price,discount,status
0,2024-06-14 15:24:38,11192,686,29.07,5.52,paid
1,2024-05-07 22:45:57,10763,1246,39.82,0.00,cancelled
2,2024-06-04 19:16:06,10966,1332,15.97,1.28,paid
3,2024-04-15 22:58:24,12368,642,32.84,5.25,placed
4,2024-06-01 10:21:20,10847,1094,16.67,1.17,paid
5,2024-06-22 22:41:19,10360,1498,35.11,2.11,paid
6,2024-04-28 19:07:09,10130,674,23.90,2.87,cancelled
7,2024-06-07 08:35:33,12825,883,28.96,1.16,paid
8,2024-05-13 21:04:39,12776,960,18.69,3.18,paid
9,2024-06-10 12:55:15,10847,1230,14.73,2.80,paid


In [93]:
# Deskriptive Statistiken des Dataframes
df.describe(include='all')


,time,customer_id,product_id,price,discount,status
count,4000,4000.000000,4000.000000,4000.000000,4000.000000,4000
unique,NaN,NaN,NaN,NaN,NaN,4
top,NaN,NaN,NaN,NaN,NaN,paid
freq,NaN,NaN,NaN,NaN,NaN,2853
mean,2024-05-16 05:35:34.792000,11506.696750,998.228500,29.795737,3.025215,NaN
min,2024-04-01 00:17:20,10001.000000,500.000000,5.790000,0.000000,NaN
25%,2024-04-22 12:21:22.249999872,10750.000000,751.000000,20.047500,1.260000,NaN
50%,2024-05-15 23:07:57.500000,11519.000000,996.000000,26.925000,2.535000,NaN
75%,2024-06-08 11:09:16.249999872,12235.000000,1251.000000,36.532500,4.240000,NaN
max,2024-06-30 23:23:12,12999.000000,1499.000000,142.670000,18.360000,NaN


#### <b>Aufgabe (2): Schreiben Sie den Inhalt der CSV-Datei 'ecommerce.csv' in die erstellte Tinyflux-Datenbank "ecommerce.db" und weisen sie Time, Fields und Tags zu.</b>

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [94]:
# Tinyflux Datenbank leeren
os.path.exists('ecommerce.db') and open('ecommerce.db','w').close()

# Einlesen der CSV-Datei in Tinyflux
with open("synthetic_ecommerce_orders.csv", mode="r") as file:
    reader = csv.DictReader(file)
    for row in reader:
            point = Point(
                time=datetime.strptime(row["time"], "%Y-%m-%d %H:%M:%S"),
                measurement="blockchain_transactions",
                fields={
                    "price": float(row["price"]),
                    "discount": float(row["discount"])
                    }, # Only numeric fields
                tags={
                    "customer_id": row["customer_id"],
                    "product_id": row["product_id"],
                    "status": row["status"] # Move non-numeric fields to tags
}
)
db.insert(point)

1

#### <b>Aufgabe (3): Die ersten drei Käufe erhalten am 7.7.2024 10% auf den am 1.4.2024 bezahlten Preis (ohne Berücksichtigung des Discounts) zurückerstattet. Fügen Sie die drei Transaktionen manuell in die Datenbank ein und weisen Sie Time, Fields und Tags gleichermassen zu, wie beim Inhalt der CSV-Datei.

</b>

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [95]:
# Berechnung der Rückerstattung am 7.7.2024

v1 = 46.79 * 0.1
print('Rückerstattung 1. Kauf:', v1)
v2 = 22.37 * 0.1
print('Rückerstattung 2. Kauf:', v2)
v3 = 38.76 * 0.1
print('Rückerstattung 3. Kauf:', v3)

Rückerstattung 1. Kauf: 4.679
Rückerstattung 2. Kauf: 2.237
Rückerstattung 3. Kauf: 3.876


| time | customer_id | product_id | price | discount | status |
| --- | --- | --- | --- | --- | --- |
| 2024-07-07 00:00:00 | 12828 | 1114 | 4.679 | 0.00 | refunded |
| 2024-07-07 00:00:00 | 12313 | 1414 | 2.237 | 0.00 | refunded |
| 2024-07-07 00:00:00 | 12117 | 1242 | 3.876 | 0.00 | refunded |

In [96]:
# Einlesen von manuellen Records
p1 = Point(
time=datetime(2024, 7, 7, 0, 0, 0, tzinfo=timezone.utc),
tags={"customer_id": "12828", "product_id": "1114", "status": "refunded"},
fields={"price": 4.679, "discount": 0}
)
p2 = Point(
time=datetime(2024, 7, 7, 0, 0, 0, tzinfo=timezone.utc),
tags={"customer_id": "12313", "product_id": "1414", "status": "refunded"},
fields={"price": 2.237, "discount": 0}
)
p3 = Point(
time=datetime(2024, 7, 7, 0, 0, 0, tzinfo=timezone.utc),
tags={"customer_id": "12117", "product_id": "1242", "status": "refunded"},
fields={"price": 3.876, "discount": 0}
)
# Insert into the DB.
db.insert_multiple([p1, p2, p3])

3

<div>

#### <b> Hinweis! 

#### Falls etwas mit der Erstellung der Datenbank nicht funktioniert, verwenden Sie für die folgenden Aufgaben die Tinyflux Datenbank 'ecommerce_fallback.db'. Gehen Sie davon aus, dass diese unter denselben, obigen Bedingungen erstellt wurde.</b>
    
#### <b>Erstellen der Verbindung mit der Tinyflux-Datenbank mit dem Namen "ecommerce_fallback.db":</b>
</div>

In [97]:
# Erstellung der Tinyflux-Datenbank (wird bereitgestellt)
db = TinyFlux("ecommerce_fallback.db")

## <b>Aufgaben: Abfragen (Queries) (Dieser Teil wird bewertet!)</b>

<b >Hinweise zu den folgenden Aufgabenstellungen:</b>
<ul>
  <li>Erstellen Sie je Aufgabe eine Time-, Field- oder Tag Abfrage bzw. eine Kombination von diesen und integrieren Sie diese in Ihren Python Code.</li>
  <li>Speichern Sie je Aufgabe die Ergebnistabelle mittels der zu Beginn dieses Notebooks bereitgestellten Funktion <b>points_to_dataframe</b> in einem DataFrame und stellen Sie diesen bzw. dessen Eigenschaften ganz oder in Teilen (z.B. erste 5 Zeilen) dar.</li>
</ul>

#### <b>Aufgabe (4): Schreiben Sie alle Bestellungen in ein Dataframe, die bis und mit dem 4.4.2024 getätigt wurden und geben Sie die ersten 10 Records dieses Dataframes aus</b>
<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [98]:
# Definieren der Query

# Datenabfrage mittels der definierten Query


# Erstellung eines Dataframes aus den Query-Resultaten und Ausgabe der ersten 10 Records

# Definieren der Query
time_query_conditions = (time <= datetime(2024, 4, 4, 0, 0, 0, tzinfo=timezone.utc))
                                          
# Datenabfrage mittels der definierten Query
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
df_result = points_to_dataframe(my_results)
df_result.head(10)

,time,customer_id,product_id,status,price,discount
0,2024-03-31 22:17:20+00:00,12313,1414,paid,22.37,3.36
1,2024-03-31 22:37:39+00:00,10758,878,placed,33.27,0.33
2,2024-03-31 23:58:15+00:00,11028,1166,paid,13.18,1.05
3,2024-04-01 01:33:13+00:00,12496,1414,paid,40.91,3.27
4,2024-04-01 03:18:39+00:00,12036,1166,placed,38.58,10.03
5,2024-04-01 04:43:13+00:00,12348,1378,cancelled,46.33,5.10
6,2024-04-01 06:05:22+00:00,10611,973,refunded,47.53,6.65
7,2024-04-01 06:26:28+00:00,12512,682,paid,21.30,1.70
8,2024-04-01 06:51:58+00:00,11911,1477,paid,29.11,2.04
9,2024-04-01 07:07:05+00:00,12982,1435,paid,19.27,0.00


#### <b>Aufgabe (5): Ermitteln Sie über den gesamten Datensatz die Anzahl Transaktionen pro Monat</b>

<b>Details zur Aufgabenstellung:</b>
- Die Anzahl Transaktionen kann mittels len(df_result), angewendet auf das Dataframe oder die Rohausgabe des Queries, herausgegeben werden

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [99]:
# Definieren der Queries
# 01/24
q1 = (time >= datetime(2024, 4, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 5, 31, tzinfo=timezone.utc))
resq1 = db.search(q1)
df_result_q1 = points_to_dataframe(resq1)




In [100]:
# Definieren der Queries
# 01/24
#q1 = (time >= datetime(2024, 1, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 1, 31, tzinfo=timezone.utc))
# 02/24
#q2 = (time >= datetime(2024, 2, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 2, 29, tzinfo=timezone.utc))
# 03/24
#q3 = (time >= datetime(2024, 3, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 3, 31, tzinfo=timezone.utc))
# 04/24
q4 = (time >= datetime(2024, 4, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 4, 30, tzinfo=timezone.utc))
# 05/24
q5 = (time >= datetime(2024, 5, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 5, 31, tzinfo=timezone.utc))
# 06/24
q6 = (time >= datetime(2024, 6, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 6, 30, tzinfo=timezone.utc))
# 07/24
q7 = (time >= datetime(2024, 7, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 7, 31, tzinfo=timezone.utc))
# 08/24
q8 = (time >= datetime(2024, 8, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 8, 31, tzinfo=timezone.utc))
# 09/24
q9 = (time >= datetime(2024, 9, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 9, 30, tzinfo=timezone.utc))
# 10/24
q10 = (time >= datetime(2024, 10, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 10, 31, tzinfo=timezone.utc))
# 11/24
q11 = (time >= datetime(2024, 11, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 11, 30, tzinfo=timezone.utc))
# 12/24
q12 = (time >= datetime(2024, 12, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 12, 31, tzinfo=timezone.utc))

# Datenabfrage mittels der definierten Queries
my_results_q1 = db.search(q1)
my_results_q2 = db.search(q2)
my_results_q3 = db.search(q3)
my_results_q4 = db.search(q4)
my_results_q5 = db.search(q5)
my_results_q6 = db.search(q6)
my_results_q7 = db.search(q7)
my_results_q8 = db.search(q8)
my_results_q9 = db.search(q9)
my_results_q10 = db.search(q10)
my_results_q11 = db.search(q11)
my_results_q12 = db.search(q12)


# Erstellung eines Dataframes aus den Query-Resultaten
#df_result_q1 = points_to_dataframe(my_results_q1)
#df_result_q2 = points_to_dataframe(my_results_q2)
#df_result_q3 = points_to_dataframe(my_results_q3)
df_result_q4 = points_to_dataframe(my_results_q4)
df_result_q5 = points_to_dataframe(my_results_q5)
df_result_q6 = points_to_dataframe(my_results_q6)
df_result_q7 = points_to_dataframe(my_results_q7)
#df_result_q8 = points_to_dataframe(my_results_q8)
#df_result_q9 = points_to_dataframe(my_results_q9)
#df_result_q10 = points_to_dataframe(my_results_q10)
#df_result_q11 = points_to_dataframe(my_results_q11)
#df_result_q12 = points_to_dataframe(my_results_q12)


# Ausgabe der aggregierten Resultate
print(f"April 24 {len(df_result_q4)}")
print(f"Mai 24 {len(df_result_q5)}")
print(f"Juni 24 {len(df_result_q6)}")
print(f"Juli 24 {len(df_result_q7)}")






April 24 1286
Mai 24 1307
Juni 24 1281
Juli 24 3


#### <b>Aufgabe (6): Ermitteln Sie den Median aller Discount-Beträge pro Status und Monat</b>

<b>Details zur Aufgabenstellung:</b>
- Alle im Datensatz vorhandenen Status können Sie mittels der Pandas-Funktion "df.attributname.unique()" (https://pandas.pydata.org/docs/reference/api/pandas.unique.html) ermitteln
- Den Durchschnitt über eine Spalte können Sie mittels der Pandas-Funktion df[['spaltenname']].mean() https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html ermitteln

<b style="color: gray;">(max. erreichbare Punkte: 12)</b>

In [101]:
df.status.unique()


array(['paid', 'cancelled', 'placed', 'refunded'], dtype=object)

In [114]:
qq4 = (time >= datetime(2024, 4, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 4, 30, tzinfo=timezone.utc))
# 05/24
qq5 = (time >= datetime(2024, 5, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 5, 31, tzinfo=timezone.utc))
# 06/24
qq6 = (time >= datetime(2024, 6, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 6, 30, tzinfo=timezone.utc))
# 07/24
qq7 = (time >= datetime(2024, 7, 1, tzinfo=timezone.utc)) & (time <= datetime(2024, 7, 31, tzinfo=timezone.utc))

# Definieren der Tag-Queries
tag1 = (tags.status == 'paid')
tag2 = (tags.status == 'cancelled')
tag3 = (tags.status == 'placed')
tag4 = (tags.status == 'refunded')


# Datenabfrage mittels der definierten Queries
my_results1 = db.search(tag1 & qq4 & qq5 & qq6 & qq7)
my_results11 = db.search(tag1 & qq4)
my_results12 = db.search(tag1 & qq5)
my_results13 = db.search(tag1 & qq6)
my_results14 = db.search(tag1 & qq7)

my_results2 = db.search(tag2 &  qq4 & qq5 & qq6 & qq7)
my_results21 = db.search(tag2 & qq4)
my_results22 = db.search(tag2 & qq5)
my_results23 = db.search(tag2 & qq6)
my_results24 = db.search(tag2 & qq7)

my_results3 = db.search(tag3 &  qq4 & qq5 & qq6 & qq7)
my_results31 = db.search(tag3 & qq4)
my_results32 = db.search(tag3 & qq5)
my_results33 = db.search(tag3 & qq6)
my_results34 = db.search(tag3 & qq7)

my_results4 = db.search(tag4 &  qq4 & qq5 & qq6 & qq7)
my_results41 = db.search(tag4 & qq4)
my_results42 = db.search(tag4 & qq5)
my_results43 = db.search(tag4 & qq6)
my_results44 = db.search(tag4 & qq7)

df11 = points_to_dataframe(my_results11)
df12 = points_to_dataframe(my_results12)
df13 = points_to_dataframe(my_results13)
#f14 = points_to_dataframe(my_results14)

df21 = points_to_dataframe(my_results11)
df22 = points_to_dataframe(my_results12)
df23 = points_to_dataframe(my_results13)
#df24 = points_to_dataframe(my_results14)

df31 = points_to_dataframe(my_results11)
df32 = points_to_dataframe(my_results12)
df33 = points_to_dataframe(my_results13)
#df34 = points_to_dataframe(my_results14)

df41 = points_to_dataframe(my_results11)
df42 = points_to_dataframe(my_results12)
df43 = points_to_dataframe(my_results13)
#df44 = points_to_dataframe(my_results14)


print(f"Für Staus paid und April {df11[['discount']].mean()}")
print(f"Für Staus paid und Mai {df12[['discount']].mean()}")
print(f"Für Staus paid und Juni {df13[['discount']].mean()}")

print(f"Für Staus cancelled und April {df21[['discount']].mean()}")
print(f"Für Staus cancelled und Mai {df12[['discount']].mean()}")
print(f"Für Staus cancelled und Juni {df13[['discount']].mean()}")

print(f"Für Staus placed und April {df31[['discount']].mean()}")
print(f"Für Staus placed und Mai {df32[['discount']].mean()}")
print(f"Für Staus placed und Juni {df33[['discount']].mean()}")

print(f"Für Staus refunded und April {df41[['discount']].mean()}")
print(f"Für Staus refunded und Mai {df42[['discount']].mean()}")
print(f"Für Staus refunded und Juni {df43[['discount']].mean()}")






Für Staus paid und April discount    3.05048
dtype: float64
Für Staus paid und Mai discount    3.027086
dtype: float64
Für Staus paid und Juni discount    3.097709
dtype: float64
Für Staus cancelled und April discount    3.05048
dtype: float64
Für Staus cancelled und Mai discount    3.027086
dtype: float64
Für Staus cancelled und Juni discount    3.097709
dtype: float64
Für Staus placed und April discount    3.05048
dtype: float64
Für Staus placed und Mai discount    3.027086
dtype: float64
Für Staus placed und Juni discount    3.097709
dtype: float64
Für Staus refunded und April discount    3.05048
dtype: float64
Für Staus refunded und Mai discount    3.027086
dtype: float64
Für Staus refunded und Juni discount    3.097709
dtype: float64


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [115]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('IP Address:', socket.gethostbyname(socket.gethostname()))
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1030-azure
Datetime: 2025-10-06 08:35:56
Python Version: 3.12.1
IP Address: 127.0.0.1
-----------------------------------
